In [4]:
import numpy as np

In [12]:
sound_speed = 340.29

In [8]:
def convert_ft_to_m(feets):
    return feets * 0.3048

In [9]:
convert_ft_to_m(18)

5.486400000000001

In [2]:
def pixel_distance(l, h):
    return np.sqrt(l**2 + h**2)

In [17]:
#double echo time delay
d_1 = 8
d_2 = 11.79
print("First Echo after : {:.5}s".format(2 * (d_1 / sound_speed)))
print("First Echo after : {:.5}s".format(2 * (d_2 / sound_speed)))

First Echo after : 0.047019s
First Echo after : 0.069294s


In [18]:
sound_speed * 0.016

5.444640000000001

In [20]:
(d_2 - d_1) * 2

7.579999999999998

In [5]:
def v(d, t):
    return d/t

def convert_ms_to_kmh(v):
    return v * 3.6

In [6]:
d1_p = pixel_distance(2171, 14)
d1_r = 5.4864
d2_p = pixel_distance(4633, 33)
d = (d2_p * d1_r) / d1_p
d = 11.70
t = 0.332
convert_ms_to_kmh(v(d, t))

126.86746987951805

In [13]:
(2.3 / 340.29)

0.006758940903347144

In [10]:
"{:.7f}".format(1/44100)

'0.0000227'